In [4]:
import numpy as np
import pandas as pd
import pickle

In [6]:
def remove_NaN(d) :  
    """ Remove rows of a DataFrame with missing firing rate """
    M = np.stack(d['Spike rate'])
    NaN_rows = np.isnan(M).any(axis=1)
    return d.iloc[~NaN_rows,:]


def produce_trial_index(d) :
    """Produce a dictionnary of trial indices corresponding to each label."""
    
    trial_idx = None
    
    go_hit_idx = (d['Choice'] == 1)&(d['Category ID'] == 2)&(d['Context'] == 0)
    go_error_idx = (d['Choice'] == 1)&(d['Category ID'] == 1)&(d['Context'] == 0)

    nogo_hit_idx = (d['Choice'] == -1)&(d['Category ID'] == 1)&(d['Context'] == 0)
    nogo_error_idx = (d['Choice'] == -1)&(d['Category ID'] == 2)&(d['Context'] == 0)

    right_hit_idx = (d['Choice'] == 1) & (d['Category ID'] == 2)&(d['Context'] == 1)
    left_hit_idx = (d['Choice'] == -1) & (d['Category ID'] == 1)&(d['Context'] == 1)

    right_error_idx = (d['Choice'] == 1) & (d['Category ID'] == 1)&(d['Context'] == 1)
    left_error_idx = (d['Choice'] == -1) & (d['Category ID'] == 2)&(d['Context'] == 1)

    # Snooze : The animal did not respond in the 2AFC paradigm.
    snooze_right_idx = (d['Choice'] == 0)&(d['Category ID'] == 2)&(d['Context'] == 1)
    snooze_left_idx = (d['Choice'] == 0)&(d['Category ID'] == 1)&(d['Context'] == 1)

    trial_idx = {'Go / Go : Hit' : go_hit_idx, 'Go / Nogo : Miss':nogo_error_idx, 'NoGo / NoGo : CR' : nogo_hit_idx , \
              'NoGo / Go : FA' : go_error_idx,  'Left / Left':left_hit_idx, 'Left / Right':left_error_idx, 'Left / Snooze':snooze_left_idx, \
           'Right / Right':right_hit_idx, 'Right / Left':right_error_idx, 'Right / Snooze':snooze_right_idx}

    return trial_idx

def generate_column_label(d) :
    
    trial_idx = produce_trial_index(d)

    # Add corresponding labels to trial in each DataFrame.
    #labels = [k for k in range(1,9)]

    condition = 1
    trial_label = np.zeros(len(d))
    for idx in trial_idx.values() :
        trial_label += idx*condition
        condition += 1

    if 'Label' not in d :
        d.insert(6, 'Label', trial_label)

    # Add a column 'Label' to full DataFrames
    # 1 : Go / Go Hit,  2 : Go / Nogo : Miss,   3 :NoGo / NoGo : CR , 4: NoGo / Go : FA
    # 5 : Left / Left, 6 : Left / Right , 7 : Left / Snooze,  8 : Right / Right, 9 : Right / Left, 10 : Right / Snooze

    # Stimuli category / Animal response

    


In [8]:
files = ['M12','M13','M14','M15','M16','M17','M18','M19','M20']

df_lr = [pd.read_pickle('Pickles/' + file + '_LR') for file in files]

df_t5 = [pd.read_pickle('Pickles/' + file + '_GNG') for file in files]

data_lr = {files[i]:df_lr[i] for i in range(len(files))}
data_t5 = {files[i]:df_t5[i] for i in range(len(files))}

In [10]:
# Add time data to the dataframe. 
# Column name for all time points: "Traj"
# Column name for mean Pre-stimulus and Post-stimulus: "Pre/Post"
for file in files :
    traj_mouse_GNG = np.load('npy_extended/' + file + '_trajGNG.npy')
    traj_mouse_LR = np.load('npy_extended/' + file + '_trajLR.npy')
    data_t5[file]['Traj'] = list(traj_mouse_GNG) 
    data_lr[file]['Traj'] = list(traj_mouse_LR)
    
    T = np.shape(traj_mouse_GNG)[1]
    
    pre_post_GNG = np.array([np.mean(traj_mouse_GNG[:,:T//2,:],axis=1),np.mean(traj_mouse_GNG[:,T//2:,:],axis=1)])
    pre_post_GNG = np.swapaxes(pre_post_GNG,0,1)
    
    pre_post_LR = np.array([np.mean(traj_mouse_LR[:,:T//2,:],axis=1),np.mean(traj_mouse_LR[:,T//2:,:],axis=1)])
    pre_post_LR = np.swapaxes(pre_post_LR,0,1)
    
    data_t5[file]['Pre/Post'] = list(pre_post_GNG)
    data_lr[file]['Pre/Post'] = list(pre_post_LR)

In [12]:
# Correct for data format
for file in files :
    data_t5[file] = remove_NaN(data_t5[file])
    data_lr[file] = remove_NaN(data_lr[file])

    data_t5[file]['Reward'] = data_t5[file]['Reward'].fillna(0)
    
    data_t5[file]['Choice'] = 2*data_t5[file]['Choice'] - 1
    data_lr[file]['Choice'] = (2*data_lr[file]['Choice'] - 3)*(data_lr[file]['Choice'] != 0)
    
    if 'Context' not in data_lr[file] :
        data_lr[file].insert(8, 'Context', np.ones(len(data_lr[file])))
        
    if 'Context' not in data_t5[file] :
        data_t5[file].insert(8, 'Context', np.zeros(len(data_t5[file])))
        
data_full = {file:pd.concat([data_t5[file],data_lr[file]],ignore_index=True) for file in files} 

In [13]:
# Add a column "Motor action" describing if the animal licked or not during the trial.
for file in files :
    d = data_full[file]
    d_t5 = data_t5[file]
    d_lr = data_lr[file]
    
    motor_action_t5 = d_t5['Choice'].astype(float)
    
    motor_action_lr = (d_lr['Go right'] + d_lr['Go left']).astype(float)
    
    motor_action_full = pd.concat([motor_action_t5,motor_action_lr],ignore_index = True)
    
    d.insert(8,'Motor action', motor_action_full)

In [16]:
# Label each trial with its corresponding condition.
# We have a total of 2*2 + 3*2 = 10 conditions for both tasks (see produce_trial_index function).
for file in files :
    generate_column_label(data_full[file])


In [17]:
with open('Dataframes/df_full.pickle', 'wb') as f:
    pickle.dump(data_full, f)

with open('Dataframes/df_GNG.pickle', 'wb') as f:
    pickle.dump(data_t5, f)

with open('Dataframes/df_LR.pickle', 'wb') as f:
    pickle.dump(data_lr, f)